In [1]:
import numpy as np
import matplotlib.pyplot as plt

from cytools import Polytope
from cytools.triangulation import Triangulation
from integer_rref import i4mat_rref


Info: A more recent version of CYTools is available: v1.0.4 -> v1.0.7.
We recommend upgrading before continuing.
On Linux and macOS you can update CYTools by running 'cytools --update'
and on Windows you can do this by running the updater tool.



In [15]:
def get_indices(p, subp):
    return [np.where(np.all(v == p, axis=1))[0][0] for v in subp]

def boundary(simplices):
    dsimplices = []
    for s in simplices:
        for i in range(1, len(s)):
            dsimplices.append(
                np.append(s[:i], s[(i+1):]))

    dsimplices = np.asarray(dsimplices)
    return dsimplices

def restrict(dsimplices, subp_vertices):
    sub_simplices = np.where(np.all(np.isin(dsimplices, subp_vertices), axis=1))[0]
    sub_simplices = dsimplices[sub_simplices]
    
    local_sub_simplices = []
    for s in sub_simplices:
        local_s = []
        for v in s:
            local_s.append(np.where(subp_vertices == v)[0][0])
        local_sub_simplices.append(local_s)
    return np.unique(local_sub_simplices, axis=0)

def project(vertices):
    vertices_copy = np.array(vertices, copy=True)
    W = np.asarray(i4mat_rref(vertices.shape[0], vertices.shape[1], vertices_copy)[0]).astype(np.float64)
    local_vertices = np.round(vertices@np.linalg.pinv(W))
    idx = np.argwhere(np.all(local_vertices[..., :] == 0, axis=0))
    
    return np.delete(local_vertices, idx, axis=1)

In [4]:
p = np.asarray([[ 0,  0,  0],
   [-1, -1,  0],
   [-1,  1,  0],
   [ 2, -1,  0],
   [ 0,  0, -1],
   [ 0,  0,  1],
   [-1,  0,  0],
   [ 0, -1,  0],
   [ 1, -1,  0]])


subp_1 = np.asarray([[ 0,  0, 0],
   [-1, -1, 0],
   [-1,  1, 0],
   [ 2, -1, 0]])
subp_1 = Polytope(subp_1).points()

subp_2 = np.array([
    [0, 0, 0],
    [-1,  1, 0],
    [ 2, -1, 0],
    [ 0, -1, 0]])

subp_2 = Polytope(subp_2).points()

In [7]:
simplices = Polytope(p).triangulate().simplices()

In [21]:
subsimplices_1 = restrict(
    dsimplices    = boundary(simplices),
    subp_vertices = get_indices(p, subp_1))

subsimplices_2 = restrict(
    dsimplices    = boundary(simplices),
    subp_vertices = get_indices(p, subp_2))

In [23]:
Triangulation(
    triang_pts = project(subp_1),
    simplices = subsimplices_1)

A fine, regular, star triangulation of a 2-dimensional point configuration with 7 points in ZZ^2